In [157]:
# importamos las dependencias
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    make_scorer
)

[nltk_data] Downloading package punkt to /Users/trabajo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/trabajo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/trabajo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [158]:
starter_df = pd.read_csv("../data/youtoxic_english_1000.csv")
starter_df = starter_df[["Text", "IsToxic"]]

starter_df.head(5)

,Text,IsToxic
0,If only people would just take a step back and...,False
1,Law enforcement is not trained to shoot to app...,True
2,\nDont you reckon them 'black lives matter' ba...,True
3,There are a very large number of people who do...,False
4,"The Arab dude is absolutely right, he should h...",False


In [159]:
starter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     1000 non-null   object
 1   IsToxic  1000 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 8.9+ KB


In [160]:
count_hate = (starter_df["IsToxic"] == True).sum()
count_no_hate = (starter_df["IsToxic"] == False).sum()
print(f"Hay {count_hate} comentarios de odio")
print(f"Hay {count_no_hate} comentarios sin odio")

Hay 462 comentarios de odio
Hay 538 comentarios sin odio


In [161]:
new_df = pd.read_csv("../data/labeled_data.csv")

new_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [162]:
count = (new_df["hate_speech"] == 0).sum()
print(f"Hay {count} comentarios de odio en el nuevo dataset")

Hay 19790 comentarios de odio en el nuevo dataset


In [163]:
nuevo_df = new_df[new_df["hate_speech"] == 0][["tweet", "hate_speech"]]
nuevo_df = nuevo_df.rename(columns={
    "tweet": "Text",
    "hate_speech": "IsToxic"
})
nuevo_df["IsToxic"] = nuevo_df["IsToxic"] == 0

nuevo_df = nuevo_df.sample(n=500, random_state=42)

nuevo_df.to_csv("../data/extra_data.csv", index=False)

print(nuevo_df.head(20))

                                                    Text  IsToxic
4349   @OptimusOdd mine will say "the intimidator" &#...     True
21677  There are girls. There are women. There are la...     True
6639                  @marshabitch don't start lil bitch     True
19270  RT @ispeak_sarcasms: the nigga with no future ...     True
12819  Meek do some nut shit one time he a bitch I gu...     True
18407  RT @__brookenicole: How many bitches have actu...     True
18290  RT @_ImtrILLasFuk: A girl KNOWS when a bitch l...     True
23668  bitches corny af you claim to be happy but cry...     True
8820   Don't talk about Homecoming until you actually...     True
9895   Horny mature babe in a kimono has her pussy fi...     True
20213  RT @tr4pb0y: when the pussy so good &amp; u fo...     True
13896  Practically got arrested tonight, not allowed ...     True
16028  RT @KingCuh: ouuua ita siana. alu polosi ho ni...     True
20801  Should I be worried that a lady behind my buil...     True
983    &#1

In [164]:
nuevo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 4349 to 16745
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     500 non-null    object
 1   IsToxic  500 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 8.3+ KB


In [165]:
extra_df = pd.read_csv("../data/extra_data.csv")
df = pd.concat([starter_df, extra_df], ignore_index=True)

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     1500 non-null   object
 1   IsToxic  1500 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 13.3+ KB


In [167]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def limpiar_texto(texto):
    tokens = nltk.word_tokenize(str(texto).lower())
    tokens = [t for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

df['texto_limpio'] = df['Text'].apply(limpiar_texto)

vectorizer = TfidfVectorizer(
    max_features=170,
    min_df=35,
    max_df=0.6,
    ngram_range=(1, 2),
    sublinear_tf=True
)
X = vectorizer.fit_transform(df['texto_limpio'])
y = df['IsToxic'].astype(int)

modelo = LogisticRegression(C=0.055, max_iter=1500, class_weight="balanced")
f1_scorer = make_scorer(f1_score, pos_label=1)
scores = cross_val_score(modelo, X, y, cv=5, scoring=f1_scorer)

print(f"\n=== Validación cruzada (F1, clase tóxica) ===")
print(f"F1-score promedio: {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelo.fit(X_train, y_train)

y_train_pred = modelo.predict(X_train)
print("\n=== Evaluación en TRAIN ===")
print(confusion_matrix(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))

y_test_pred = modelo.predict(X_test)
print("\n=== Evaluación en TEST ===")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

f1_train = f1_score(y_train, y_train_pred, pos_label=1)
f1_test = f1_score(y_test, y_test_pred, pos_label=1)

overfitting_f1_pct = ((f1_train - f1_test) / f1_train) * 100 if f1_train != 0 else 0.0
print(f"F1-score TRAIN (clase tóxica): {f1_train:.3f}")
print(f"F1-score TEST  (clase tóxica): {f1_test:.3f}")
print(f"Overfitting (basado en F1): {overfitting_f1_pct:.2f}%")

def predict_true_false(prob):
    return prob >= 0.5

def predecir_toxicidad(texto):
    texto_limpio = limpiar_texto(texto)
    texto_vect = vectorizer.transform([texto_limpio])
    prob = modelo.predict_proba(texto_vect)
    print(f"Probabilidades de cada clase [no tóxico, tóxico]: {prob}")
    prob_toxico = prob[0][1]
    print(f"Probabilidad de toxicidad: {prob_toxico:.3f}")
    prediccion_final = predict_true_false(prob_toxico)
    return {
        "prediccion": prediccion_final,
        "probabilidad_toxico": round(prob_toxico, 3)
    }


=== Validación cruzada (F1, clase tóxica) ===
F1-score promedio: 0.576
Desviación estándar: 0.311

=== Evaluación en TRAIN ===
[[417  13]
 [326 444]]
              precision    recall  f1-score   support

           0       0.56      0.97      0.71       430
           1       0.97      0.58      0.72       770

    accuracy                           0.72      1200
   macro avg       0.77      0.77      0.72      1200
weighted avg       0.82      0.72      0.72      1200


=== Evaluación en TEST ===
[[100   8]
 [ 86 106]]
              precision    recall  f1-score   support

           0       0.54      0.93      0.68       108
           1       0.93      0.55      0.69       192

    accuracy                           0.69       300
   macro avg       0.73      0.74      0.69       300
weighted avg       0.79      0.69      0.69       300

F1-score TRAIN (clase tóxica): 0.724
F1-score TEST  (clase tóxica): 0.693
Overfitting (basado en F1): 4.27%


In [168]:
print(predecir_toxicidad("stupid"))

Probabilidades de cada clase [no tóxico, tóxico]: [[0.4544119 0.5455881]]
Probabilidad de toxicidad: 0.546
{'prediccion': np.True_, 'probabilidad_toxico': np.float64(0.546)}
